# TP8 : Algorithme du gradient projeté à pas fixe
Dans ce TP, nous nous intéressons à l'algorithme de gradient projeté à pas fixe pour la résolution du problème:
$$\min_{x\in\mathbb{R}^n} f(x) \quad \mbox{ sous la contrainte: } x\in C,$$
où $C$ est un sous-ensemble convexe, non vide de $\mathbb{R}^n$.

Le but de cette séance est l'écriture d'un code de minimisation locale, et l'évaluation de ses performances sur les fonctions tests suivantes:

* $f(x,y,z) = (x+y-1)^2+(y-z)^2+(z-\frac{1}{2})^2$.
* $g(x,y,z) = x+2y+3z$.

pour les ensembles de contraintes suivants:

* $C_1 = \{X\in\mathbb{R}^3 : \|X\|=1\}$ sphère unité de $\mathbb{R}^3$.
* $C_2 = \{X=(x,y,z)\in\mathbb{R}^3 : -1\leq x,y,z\leq 1\}$ cube dans $\mathbb{R}^3$.
* $C_3 = \{X=(x,y,z)\in\mathbb{R}^3 : x+y+z=0\}$ plan dans $\mathbb{R}^3$.


>**Rappel :** on appelle `oracle` une routine qui à un `x` donné, renvoie la valeur $f(x)$ du critère et le gradient $\nabla f(x)$ (ou une approximation du gradient) s'il existe:

$$[f(x),\nabla f(x)] = \textrm{oracle}(x).$$

## Structure de l'algorithme de gradient projeté à pas fixe

* **Données:** $X_0\in \mathbb{R}^n$ point initial arbitraire, un oracle, un opérateur de projection sur $C$, $s>0$ pas fixe arbitraire.
* **Initialisation:** Numéro d'itération: $k=0$.
* **Tant que** le critère d'arrêt n'est pas satisfait, **faire**
  1. Calcul d'une itération du gradient à pas fixe à partir de $X_k$:
    $$Y_{k+1} = X_k-s\nabla f(X_k).$$  
  2. Calcul de la projection de $Y_{k+1}$ sur $C$.
  3. Mise à jour du prochain itéré $X_{k+1}$.
  4. $k=k+1$.



Pour chacun des ensembles de contraintes proposés, nous voulons résoudre les deux problèmes-tests de minimisation:
$$(P_i)\quad\quad \min_{X=(x,y,z)\in\mathbb{R}^3} f(x,y,z)\quad\mbox{ sous la contrainte: }\quad X\in C_i,\quad i\in \{1,2,3\}$$
$$(Q_i)\quad\quad \min_{X=(x,y,z)\in\mathbb{R}^3} g(x,y,z)\quad\mbox{ sous la contrainte: }\quad X\in C_i,\quad i\in \{1,2,3\}$$

> **A faire :** Définir et implémenter les trois opérateurs de projection sur les ensembles $C_1$, $C_2$ et $C_3$.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as npl

In [22]:
import numpy as np
def projection1(x):
    x2 = x**2
    l = np.sqrt(sum(x2))
    if l > 1:
        y = x/l
    else:
        y = x
    y = np.array(y)
    return y
def projection2(x):
    y = [0,0,0]
    if x[0] != 0:
        y[0] = (x[0]/abs(x[0]))*min(abs(x[0]),1)
    else:
        y[0] = 0
    if x[1] != 0:
        y[1] = (x[1]/abs(x[1]))*min(abs(x[1]),1)
    else:
        y[1] = 0
    if y[2] != 0:
        y[2] = (x[2]/abs(x[2]))*min(abs(x[2]),1)
    else: 
        y[2] = 0
    y = np.array(y)
    return y
def projection3(x):
    s = sum(x)/3
    y = [0,0,0]
    y = x - s
    y = np.array(y)
    return y

>**A faire :** Calculer et implémenter les oracles pour les deux fonctions proposées.

In [3]:
def oracle1(x):
    f = (x[0]+x[1]-1)**2+(x[1]-x[2])**2+(x[2]-1/2)**2
    df = [2*(x[0]+x[1]-1),2*(x[0]+2*x[1]-x[2]-1),4*x[2]-2*x[1]-1]
    df = np.array(df)
    return f,df
def oracle2(x):
    f = x[0] + 2*x[1] +3*x[2]
    df = [1,2,3]
    df = np.array(df)
    return f,df

>**A faire** : Définir un critère d'arrêt acceptable pour l'algorithme de gradient projeté à pas fixe et écrire la routine complète de minimisation par l'algorithme de gradient projeté à pas fixe.

In [10]:
def gradproj(oracle,projection,xini=[1,0,0],h=0.1):
    f,df = oracle(xini)
    df = np.array(df)
    xiter = [xini]
    iteration = 0
    sdf2 = sum([t**2 for t in df])
    while (iteration < 50000) and (sdf2 > 0.01):
        x = xiter[-1] - h*df
        xiter += [x]
        x = projection(x)
        xiter += [x]
        f,df = oracle(x)
        df = np.array(df)
        sdf2 = sum([t**2 for t in df])
        iteration += 1
    return x,xiter,iteration
x,xiter,nbiter = gradproj(oracle1,projection1)

>**A faire**: Résoudre numériquement les six problèmes $(P_i)$ et $(Q_i)$, $i\in \{1,2,3\}$. Commenter et interpréter les résultats obtenus.

In [23]:
#1
r1 = gradproj(oracle1,projection1)
print(r1[0],r1[2])
#2
r2 = gradproj(oracle1,projection2)
print(r2[0],r2[2])
#3
r3 = gradproj(oracle1,projection3)
print(r3[0],r3[2])
#4
r4 = gradproj(oracle2,projection1)
print(r4[0],r4[2])
#5
r5 = gradproj(oracle2,projection2)
print(r5[0],r5[2])
#6
r6 = gradproj(oracle2,projection3)
print(r6[0],r6[2])

[0.68513956 0.35152504 0.41759945] 29
[1. 0. 0.] 50000
[ 0.5  -0.25 -0.25] 50000
[-0.26726124 -0.53452248 -0.80178373] 50000
[-1. -1.  0.] 50000
[ 5.00066667e+03 -3.33333332e-01 -5.00033333e+03] 50000
